In [451]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [452]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [453]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [454]:
'''
    Функция навигации по данным из api response
    возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [455]:
'''
    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
    и вывод даты в строку

    delta = datetime.timedelta(days=1)
    day_current = datetime.date.fromisoformat(start_date) + delta
    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

"\n    Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)\n    и вывод даты в строку\n\n    delta = datetime.timedelta(days=1)\n    day_current = datetime.date.fromisoformat(start_date) + delta\n    date_str = datetime.date.strftime(day_current, '%Y-%m-%d')\n"

In [456]:
# Делаем список данных за n дней

items_list = []
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']

# n - количество запросов(дней)
n = 1

for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_days = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_days:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data'][0]
        dict_relative_velocity = dict_close_approach_data['relative_velocity']
        dict_miss_distance = dict_close_approach_data['miss_distance']

        # Чистим словарь от вложенных словарей
        for key in keys:
            data_day_item.pop(key, None)
        data_day_item_short = data_day_item

        df1 = pd.DataFrame([data_day_item_short])
        df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
        df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
        df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
        df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)

        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})
        
        
        
        dict_item = df_item.to_dict()
        keys_list = list(dict_item.keys())
        dff = pd.DataFrame(columns=keys_list)

        dff = pd.concat([dff, df_item])
dff

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers
0,54330582,(2022 WQ9),28.73,False,False,0.00477,0.010667,2020-07-08,2020-Jul-08 08:48,1594198080000,NaN,Earth,5.4029424309,19450.5927510908,24512180.21939535


In [457]:
# Навигация по ссылкам
# Работаем с данными за один день 2020-07-07
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,15,NaN
2020-07-08,NaN,15,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [458]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2242450?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '2242450',
 'neo_reference_id': '2242450',
 'name': '242450 (2004 QY2)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2242450',
 'absolute_magnitude_h': 14.55,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 3.2700543928,
   'estimated_diameter_max': 7.3120639125},
  'meters': {'estimated_diameter_min': 3270.0543928193,
   'estimated_diameter_max': 7312.0639124658},
  'miles': {'estimated_diameter_min': 2.0319169681,
   'estimated_diameter_max': 4.5435044654},
  'feet': {'estimated_diameter_min': 10728.5252541373,
   'estimated_diameter_max': 23989.7117665742}},
 'is_potentially_hazardous_asteroid': True,
 'close_approach_data': [{'close_approach_date': '2020-07-08',
   'close_approach_date_full': '2020-Jul-08 11:27',
   'epoch_date_close_approach': 1594207620000,
   'relative_velocity': {'kilometers_per_second': '26.6793702848',
    'ki

In [459]:
# Создаем словари с выбранными данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data']
dict_relative_velocity = dict_close_approach_data[0]['relative_velocity']
dict_miss_distance = dict_close_approach_data[0]['miss_distance']

In [460]:
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']
for key in keys:
    data_day_item.pop(key, None)
data_day_item_short = data_day_item
data_day_item_short

{'id': '2242450',
 'neo_reference_id': '2242450',
 'name': '242450 (2004 QY2)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2242450',
 'absolute_magnitude_h': 14.55,
 'is_potentially_hazardous_asteroid': True,
 'is_sentry_object': False}

In [461]:
# Создаем DataFrame
# Объединяем DataFrame
#df1 = pd.DataFrame(data_day_item_short, index=[0])
df2 = pd.DataFrame(dict_estimated_diameter, index=[0])
#df3 = pd.DataFrame(dict_close_approach_data, index=[0])
#df4 = pd.DataFrame(dict_relative_velocity, index=[0])
#df5 = pd.DataFrame(dict_miss_distance, index=[0])
#df1.join(df2).join(df3).join(df4).join(df5)
type(df2)

pandas.core.frame.DataFrame

In [462]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                  'neo_reference_id',
                                                  'miss_distance',
                                                  'miles_per_hour',
                                                  'astronomical',
                                                  'lunar',
                                                  'miles'
                                                  ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                  'estimated_diameter_min': 'estimated_diameter_km_min',
                  'estimated_diameter_max': 'estimated_diameter_km_max'})
df_item.to_dict()

{'id': {0: '54330582'},
 'name': {0: '(2022 WQ9)'},
 'absolute_magnitude_h': {0: 28.73},
 'is_potentially_hazardous_asteroid': {0: False},
 'is_sentry_object': {0: False},
 'estimated_diameter_km_min': {0: 3.2700543928},
 'estimated_diameter_km_max': {0: 7.3120639125},
 'close_approach_date': {0: '2020-07-08'},
 'close_approach_date_full': {0: '2020-Jul-08 08:48'},
 'epoch_date_close_approach': {0: 1594198080000},
 'relative_velocity': {0: nan},
 'orbiting_body': {0: 'Earth'},
 'kilometers_per_second': {0: '5.4029424309'},
 'kilometers_per_hour': {0: '19450.5927510908'},
 'miss_distance_kilometers': {0: '24512180.21939535'}}